#  ANOVA

In statistics, **Analysis of Variance (ANOVA)** is also used to analyze the differences among group means. The difference between t-test and ANOVA is the former is ued to compare two groups whereas the latter is used to compare three or more groups. [Read more about the difference between t-test and ANOVA](http://b.link/anova24).

From the ANOVA test, you receive two numbers. The first number is called the **F-value** which indicates whether your null-hypothesis can be rejected. The critical F-value that rejects the null-hypothesis varies according to the number of total subjects and the number of subject groups in your experiment. In [this table](http://b.link/eda14) you can find the critical values of the F distribution. **If you are confused by the massive F-distribution table, don't worry. Skip F-value for now and study it at a later time. In this challenge you only need to look at the p-value.**

The p-value is another number yielded by ANOVA which already takes the number of total subjects and the number of experiment groups into consideration. **Typically if your p-value is less than 0.05, you can declare the null-hypothesis is rejected.**

In this challenge, we want to understand whether there are significant differences among various types of pokemons' `Total` value, i.e. Grass vs Poison vs Fire vs Dragon... There are many types of pokemons which makes it a perfect use case for ANOVA. 

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Load the data:
pokemon = pd.read_csv('pokemon.txt')
pokemon

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,719,Diancie,Rock,Fairy,600,50,100,150,100,150,50,6,True
796,719,DiancieMega Diancie,Rock,Fairy,700,50,160,110,160,110,110,6,True
797,720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70,6,True
798,720,HoopaHoopa Unbound,Psychic,Dark,680,80,160,60,170,130,80,6,True


**To achieve our goal, we use three steps:**

1. **Extract the unique values of the pokemon types.**

1. **Select dataframes for each unique pokemon type.**

1. **Conduct ANOVA analysis across the pokemon types.**

#### First let's obtain the unique values of the pokemon types. These values should be extracted from Type 1 and Type 2 aggregated. Assign the unique values to a variable called `unique_types`.

*Hint: the correct number of unique types is 19 including `NaN`. You can disregard `NaN` in next step.*

In [3]:
type1_counts = pokemon['Type 1'].value_counts()

# Convert it into a DataFrame
type1_df = pd.DataFrame({'Type 1': type1_counts.index, 'Count1': type1_counts.values})
# Print the resulting DataFrame
type1_df

,Type 1,Count1
0,Water,112
1,Normal,98
2,Grass,70
3,Bug,69
4,Psychic,57
5,Fire,52
6,Electric,44
7,Rock,44
8,Ground,32
9,Ghost,32


In [4]:
# Your code here
type2_counts = pokemon['Type 2'].value_counts()

# Convert it into a DataFrame
type2_df = pd.DataFrame({'Type 2 ': type2_counts.index, 'count2': type2_counts.values})
type2_df

,Type 2,count2
0,Flying,97
1,Ground,35
2,Poison,34
3,Psychic,33
4,Fighting,26
5,Grass,25
6,Fairy,23
7,Steel,22
8,Dark,20
9,Dragon,18


In [5]:
unique_types=pd.concat([type1_df,type2_df],axis=1)
unique_types

,Type 1,Count1,Type 2,count2
0,Water,112,Flying,97
1,Normal,98,Ground,35
2,Grass,70,Poison,34
3,Bug,69,Psychic,33
4,Psychic,57,Fighting,26
5,Fire,52,Grass,25
6,Electric,44,Fairy,23
7,Rock,44,Steel,22
8,Ground,32,Dark,20
9,Ghost,32,Dragon,18


In [6]:
type1= pokemon['Type 1'].unique()
type1 = pd.DataFrame(type1, columns=['Type'])
type2= pokemon['Type 2'].unique()
type2 = pd.DataFrame(type2, columns=['Type'])
unique_types = pd.concat([type1, type2], axis=0)
unique_types = unique_types['Type'].unique()
unique_types

array(['Grass', 'Fire', 'Water', 'Bug', 'Normal', 'Poison', 'Electric',
       'Ground', 'Fairy', 'Fighting', 'Psychic', 'Rock', 'Ghost', 'Ice',
       'Dragon', 'Dark', 'Steel', 'Flying', nan], dtype=object)

In [7]:
len(unique_types)


19

In [8]:
pokemon.head()


,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


#### Second we will create a list named `pokemon_totals` to contain the `Total` values of each unique type of pokemons.

Why we use a list instead of a dictionary to store the pokemon `Total`? It's because ANOVA only tells us whether there is a significant difference of the group means but does not tell which group(s) are significantly different. Therefore, we don't need know which `Total` belongs to which pokemon type.

*Hints:*

* Loop through `unique_types` and append the selected type's `Total` to `pokemon_groups`. Be sure to loop through BOTH `Type 1` and `Type 2` to cover all occurrances of each unique type.
* Skip the `NaN` value in `unique_types`. `NaN` is a `float` variable which you can find out by using `type()`. The valid pokemon type values are all of the `str` type.
* At the end, the length of your `pokemon_totals` should be 18.

In [9]:
# Your code here
pokemon_totals = []


In [10]:
unique_types = unique_types.tolist()


In [11]:
unique_types.remove(np.nan)

In [12]:
for type in unique_types:

    type1_pokemon = pokemon[pokemon['Type 1'] == type]
    type2_pokemon = pokemon[pokemon['Type 2'] == type]
    
    combined = pd.concat([type1_pokemon, type2_pokemon], axis=0)
    
    total_values = combined['Total'].tolist()
    
    pokemon_totals.append(total_values)

In [14]:
#print(pokemon_totals)


#### Now we run ANOVA test on `pokemon_totals`.

*Hints:*

* To conduct ANOVA, you can use `scipy.stats.f_oneway()`. Here's the [reference](http://b.link/scipy44).

* What if `f_oneway` throws an error because it does not accept `pokemon_totals` as a list? The trick is to add a `*` in front of `pokemon_totals`, e.g. `stats.f_oneway(*pokemon_groups)`. This trick breaks the list and supplies each list item as a parameter for `f_oneway`.

In [15]:
# Your code here
st.f_oneway(*pokemon_totals)


F_onewayResult(statistic=6.617538296005535, pvalue=2.6457458815984803e-15)

#### Interpret the ANOVA test result. Is the difference significant?